# Options Strategy notebook
- First step is to load all the libraries needed to properly run this notebook.

In [ ]:
import pandas as pd
import numpy as np
import eikon as ek
import json 
from business_duration import businessDuration
import scipy.interpolate

import seaborn as sns

import chart_studio as cf

from _plotly_future_ import v4_subplots
from plotly.subplots import make_subplots

import Func

# import plotly.graph_objects as go
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import plot
import ipywidgets
import plotly.offline
# plotly.offline.init_notebook_mode(connected=True)

from IPython.display import display, HTML
from IPython.display import display_html 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.simplefilter("ignore")



- Set up your key value according to the key generated at the App Key generator App on Eikon

In [2]:
value='603a61c8427642b3816e14e3dea83f1ca4dc7f7c'

ek.set_app_key(value)
ek.set_timeout(60*20)

### Choose the underlying asset (chain):
- From the chain it will bring a list of all tradable options and then you get the fields needed for the calculations.

In [3]:
chain='0#PETR4*.SA'
# chain="0#TSLA*.U"
OptionsList, err = ek.get_data(chain,'RData')
OptionsList=OptionsList.Instrument[~OptionsList.Instrument.isnull()].values

In [4]:
len(OptionsList)

1203

In [5]:
# optionsInfo, err = ek.get_data(list(OptionsList),['TRADE_DATE','TRDPRC_1',"PUTCALLIND","EXPIR_DATE","STRIKE_PRC","VOL_X_PRC1","STOCK_RIC"])
optionsInfo, err = ek.get_data(chain,['TRADE_DATE','TRDPRC_1',"PUTCALLIND","EXPIR_DATE","STRIKE_PRC","VOL_X_PRC1","STOCK_RIC"])
optionsInfo=optionsInfo[optionsInfo.STRIKE_PRC.notnull()]
optionsInfo=optionsInfo[optionsInfo.EXPIR_DATE.astype('datetime64')>optionsInfo.TRADE_DATE.astype('datetime64').max()]
optionsInfo['VOL_X_PRC1']=optionsInfo['VOL_X_PRC1']*1000
(optionsInfo.head())


,Instrument,TRADE_DATE,TRDPRC_1,PUTCALLIND,EXPIR_DATE,STRIKE_PRC,VOL_X_PRC1,STOCK_RIC
1,PETR4R480.SA,2020-05-21,0.01,PUT,2020-06-15,4.8,20.0,PETR4.SA
2,PETR4F480.SA,None,NaN,CALL,2020-06-15,4.8,NaN,PETR4.SA
3,PETR4R580.SA,2020-05-19,NaN,PUT,2020-06-15,5.8,NaN,PETR4.SA
4,PETR4F580.SA,None,NaN,CALL,2020-06-15,5.8,NaN,PETR4.SA
5,PETR4R620.SA,2020-05-21,0.01,PUT,2020-06-15,6.2,77.0,PETR4.SA


### Colect market data
- spot price for underlying asset and risk-free interest rate

In [6]:
spot, err = ek.get_data([optionsInfo.STOCK_RIC.values[0]]+['BRSELICD=CBBR'],['TRADE_DATE','TRDPRC_1'])
# spot, err = ek.get_data([optionsInfo.STOCK_RIC.values[0]]+['USDONFSR=X'],['TRADE_DATE','TRDPRC_1'])

In [7]:
optionsInfo['Delta']=(optionsInfo.STRIKE_PRC/spot.TRDPRC_1.values[0])-1
optionsInfo_C=optionsInfo[optionsInfo.PUTCALLIND=='CALL']
optionsInfo_P=optionsInfo[optionsInfo.PUTCALLIND!='CALL']
optionsInfo_C.head()

,Instrument,TRADE_DATE,TRDPRC_1,PUTCALLIND,EXPIR_DATE,STRIKE_PRC,VOL_X_PRC1,STOCK_RIC,Delta
2,PETR4F480.SA,None,NaN,CALL,2020-06-15,4.8,NaN,PETR4.SA,-0.749609
4,PETR4F580.SA,None,NaN,CALL,2020-06-15,5.8,NaN,PETR4.SA,-0.697444
6,PETR4F620.SA,2020-05-12,NaN,CALL,2020-06-15,6.2,NaN,PETR4.SA,-0.676578
8,PETR4F380.SA,2020-05-18,NaN,CALL,2020-06-15,6.8,NaN,PETR4.SA,-0.645279
9,PETR4F740.SA,2020-05-08,NaN,CALL,2020-06-15,7.4,NaN,PETR4.SA,-0.613980


### Show all the available expiration date

In [8]:
lldate=list(set(optionsInfo.EXPIR_DATE.values))
lldate.sort()
(pd.DataFrame(lldate,columns=['Expirations']))

,Expirations
0,2020-06-15
1,2020-07-20
2,2020-08-17
3,2020-09-21
4,2020-10-19
5,2020-11-16
6,2020-12-21
7,2021-01-18
8,2021-02-22
9,2021-03-15


In [9]:
#Select Expitation -should be a list
ll=lldate[1:2]
ll

['2020-07-20']

In [10]:
# Select number of oprions around the money
numOpc=40
# Select number of oprions with % delta strik
perDelta=0.15

#### Filter according to the Delta


In [11]:
Selec_C=Func.DeltaStreikFilter(optionsInfo_C,ll,None,perDelta)
Selec_P=Func.DeltaStreikFilter(optionsInfo_P,ll,None,perDelta)

callRIC=pd.DataFrame(Selec_C).T
callRIC.columns=ll
call=pd.DataFrame([optionsInfo[optionsInfo.Instrument.isin(Selec_C[i])].STRIKE_PRC.values for i in range(len(Selec_C))]).T
call.columns=ll

putRIC=pd.DataFrame(Selec_P).T
putRIC.columns=ll
put=pd.DataFrame([optionsInfo[optionsInfo.Instrument.isin(Selec_P[i])].STRIKE_PRC.values for i in range(len(Selec_P))]).T
put.columns=ll

CSS = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(CSS))

t1,t2,call,put=Func.Screen(optionsInfo,Selec_C,Selec_P)
t1
t2

,Instrument,TRDPRC_1,STRIKE_PRC,VOL_X_PRC1
267,PETR4G46.SA,3.44,16.49,20646
269,PETR4G167.SA,3.37,16.74,73568
271,PETR4G17.SA,3.05,16.99,139122
273,PETR4G172.SA,2.94,17.24,95257
275,PETR4G19.SA,2.7,17.49,205457
278,PETR4G177.SA,2.64,17.74,40002
280,PETR4G18.SA,2.35,17.99,320187
281,PETR4G182.SA,2.28,18.24,127593
284,PETR4G20.SA,2.03,18.49,317985
286,PETR4G187.SA,1.95,18.74,139973


,Instrument,TRDPRC_1,STRIKE_PRC,VOL_X_PRC1
268,PETR4S46.SA,0.61,16.49,8695
270,PETR4S167.SA,nan,16.74,nan
272,PETR4S17.SA,0.8,16.99,61351
274,PETR4S172.SA,0.88,17.24,26925
276,PETR4S19.SA,0.91,17.49,63718
277,PETR4S177.SA,1.03,17.74,252749
279,PETR4S18.SA,1.07,17.99,170080
282,PETR4S182.SA,1.16,18.24,60910
283,PETR4S20.SA,1.28,18.49,342550
285,PETR4S187.SA,1.36,18.74,84673


# Strategies:
-----
- 1. [Bullish strategies](#Bullishstrategies)
    - 1.1 [Long Call](#Longcall)
    - 1.2 [Long Call Spread](#LongCallSpread)
    - 1.3 [Back Spread with Calls](#BackSpread)
    - 1.4 [Skip Strike Butterfly with Calls](#SkipStrike)

- 1.5 [Inverse Skip Strike Butterfly with Calls](#InverseSkip)
    - 1.6 [Christmas Tree Butterfly with Calls](#ChristmasTree)
    - 1.7 [Short Put](#ShortPut)
    - 1.8 [ Long Combination](#LongCombination)
    - 1.9 [Front Spread with Calls](#FrontSpread)

-----
- 2. [Bearish strategys](#Bearishstrategys)
    - 2.1 [Long Put](#LongPut)
    - 2.2 [Long Put Spread](#LongPutSpread)
    - 2.3 [Back Spread with Puts](#BackSpreadwithPuts)
    - 2.4 [Skip Strike Butterfly with Puts](#SkipStrikeButterflywithPuts)
    - 2.5 [Inverse Skip Strike Butterfly with Puts](#InverseSkipStrikeButterflywithPuts)
    - 2.6 [Short Call](#ShortCall)
    - 2.7 [Short Combination](#ShortCombination)
    - 2.8 [Front Spread with Puts](#FrontSpreadwithPuts)

-----
- 3. [Volatility short strategy (Sell Volatility)](#Volatilityshortstrategy(SellVolatility))
    - 3.1 [Long Butterfly Spread with Calls](#LongButterflySpreadwithCalls)
    - 3.2 [Iron Butterfly](#IronButterfly)
    - 3.3 [Long Condor Spread with Calls](#LongCondorSpreadwithCalls)
    - 3.4 [Iron Condor](#IronCondor)
    - 3.5 [Short Straddle](#ShortStraddle)
    - 3.6 [Short Strangle](#ShortStrangle)

-----
4. [Volatility Long Strategy (Buy Volatility)](#VolatilityLongStrategy(BuyVolatility))
    - 4.1 [Long Straddle](#LongStraddle)
    - 4.1 [Long Strangle](#LongStrangle)


## Options Strategies

### Bullish strategies
<a id='Bullishstrategies'>

<a id='Longcall'>

#### Long Call

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument]
data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000]
data

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='LongCallSpread'>

#### Long Call Spread

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.5),].Instrument,
             call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000,
                -1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='BackSpread'>

#### Back Spread with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
             call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000,
                2000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='SkipStrike'>

#### Skip Strike Butterfly with Calls

In [ ]:
# -- SKIP STRIKE BUTTERFLY W/CALLS
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
             call.iloc[int(len(call)*0.6),].Instrument,
             call.iloc[int(len(call)*0.8),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options
data['Amount']=[1000,
                -2000,
               1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='InverseSkip'>

#### Inverse Skip Strike Butterfly with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
             call.iloc[int(len(call)*0.6),].Instrument,
             call.iloc[int(len(call)*0.8),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000,
                2000,
               -1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='ChristmasTree'>

#### Christmas Tree Butterfly with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
             call.iloc[int(len(call)*0.7),].Instrument,
             call.iloc[int(len(call)*0.8),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000,
                -3000,
               2000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='ShortPut'>

#### Short Put

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='LongCombination'>

#### Long Combination

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.5),].Instrument,
             put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000,
                -1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='FrontSpread'>

#### Front Spread with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
             put.iloc[int(len(put)*0.6),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000,
                -2000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='Bearishstrategys'>

### Bearish strategys

<a id='LongPut'>

####  Long Put

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.6),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='LongPutSpread'>

#### Long Put Spread

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.4),].Instrument,
              put.iloc[int(len(put)*0.6),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000,
                1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='BackSpreadwithPuts'>

#### Back Spread with Puts

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.4),].Instrument,
              put.iloc[int(len(put)*0.6),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[2000,
                -1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='SkipStrikeButterflywithPuts'>

#### Skip Strike Butterfly with Puts

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.2),].Instrument,
              put.iloc[int(len(put)*0.4),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[1000,
                -2000,
               1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='InverseSkipStrikeButterflywithPuts'>

#### Inverse Skip Strike Butterfly with Puts

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.2),].Instrument,
              put.iloc[int(len(put)*0.4),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000,
                2000,
               -1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='ChristmasTreeButterflywithPuts'>

#### Christmas Tree Butterfly with Puts

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.2),].Instrument,
              put.iloc[int(len(put)*0.3),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[2000,
                -3000,
               1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='ShortCall'>

#### Short Call

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='ShortCombination'>

#### Short Combination

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.5),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None
data['Amount']=[-1000,
                1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='FrontSpreadwithPuts'>

#### Front Spread with Puts

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.5),].Instrument,
              put.iloc[int(len(put)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[-2000,
                1000]

In [ ]:
fign,p=Func.all(spot,data)
fig

<a id='Volatilityshortstrategy(SellVolatility)'>

### Volatility short strategy (Sell Volatility)

<a id='LongButterflySpreadwithCalls'>

#### Long Butterfly Spread with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.3),].Instrument,
              call.iloc[int(len(call)*0.5),].Instrument,
              call.iloc[int(len(call)*0.8),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                -2000,
               1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='IronButterfly'>

#### Iron Butterfly

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.3),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument,
              call.iloc[int(len(call)*0.5),].Instrument,
              call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                -1000,
                -1000,
               1000]
data

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='LongCondorSpreadwithCalls'>

#### Long Condor Spread with Calls

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.3),].Instrument,
              call.iloc[int(len(call)*0.4),].Instrument,
              call.iloc[int(len(call)*0.6),].Instrument,
              call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                -1000,
                -1000,
               1000]
data

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='IronCondor'>

#### Iron Condor

In [12]:
ListStrategi=[put.iloc[int(len(put)*0.3),].Instrument,
              put.iloc[int(len(put)*0.4),].Instrument,
              call.iloc[int(len(call)*0.6),].Instrument,
              call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                -1000,
                -1000,
               1000]
data

,Instrument,TRADE_DATE,TRDPRC_1,PUTCALLIND,EXPIR_DATE,STRIKE_PRC,VOL_X_PRC1,STOCK_RIC,Delta,Amount
279,PETR4S18.SA,2020-05-21,1.07,PUT,2020-07-20,17.99,170080.0,PETR4.SA,-0.061555,1000
285,PETR4S187.SA,2020-05-21,1.36,PUT,2020-07-20,18.74,84673.0,PETR4.SA,-0.022431,-1000
293,PETR4G193.SA,2020-05-21,1.38,CALL,2020-07-20,19.74,46930.0,PETR4.SA,0.029734,-1000
300,PETR4G210.SA,2020-05-21,1.04,CALL,2020-07-20,20.49,295870.0,PETR4.SA,0.068858,1000


In [13]:
fign,p=Func.all(spot,data)
fign

FigureWidget({
    'data': [{'line': {'color': 'black', 'dash': 'dash', 'width': 2},
              'name': '0 …

<a id='ShortStraddle'>

#### Short Straddle

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.5),].Instrument,
              call.iloc[int(len(call)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[-1000,
                -1000]
data

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='ShortStrangle'>

#### Short Strangle

In [ ]:
ListStrategi=[put.iloc[int(len(put)*0.3),].Instrument,
              call.iloc[int(len(call)*0.7),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[-1000,
                -1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

<a id='VolatilityLongStrategy(BuyVolatility)'>

### Volatility Long Strategy (Buy Volatility)

<a id='LongStraddle'>

#### Long Straddle

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.5),].Instrument,
              put.iloc[int(len(put)*0.5),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

In [ ]:
p

<a id='LongStrangle'>

#### Long Strangle

In [ ]:
ListStrategi=[call.iloc[int(len(call)*0.4),].Instrument,
              put.iloc[int(len(put)*0.6),].Instrument]

data=optionsInfo[optionsInfo.Instrument.isin(ListStrategi).tolist()]
pd.options.mode.chained_assignment = None

data['Amount']=[1000,
                1000]

In [ ]:
fign,p=Func.all(spot,data)
fign

## Calculation process

In [ ]:
data

In [ ]:
data['Amount']=data['Amount'].astype(float)
data['TRDPRC_1']=data['TRDPRC_1'].astype(float)
data['STRIKE_PRC']=data['STRIKE_PRC'].astype(float)
data["TRADE_DATE"]=pd.to_datetime(data["TRADE_DATE"])
data["EXPIR_DATE"]=pd.to_datetime(data["EXPIR_DATE"])

#### Volatility and Greek Calculations:

In [ ]:
ssp=spot.TRDPRC_1.values[0]
days=(data.apply(lambda row: businessDuration(startdate=row['TRADE_DATE'],enddate=row['EXPIR_DATE'],unit='day'), axis=1)).values

ImpyVola=[round(Func.ImpliedVola(spot.TRDPRC_1.values[0].astype(float),
            data["STRIKE_PRC"].values[j].astype(float),
            spot.TRDPRC_1.values[1]/100,
            .70,
            data["TRDPRC_1"].values[j].astype(float),
            days[j],
            # (((data["EXPIR_DATE"].values[j])-(data["TRADE_DATE"].values[j])).astype('timedelta64[D]').astype(int)),
            data["PUTCALLIND"].values[j],a=-12.0, b=12.0, xtol=1e-8)*100,2) for j in range(len(data))]


In [ ]:
GREEKS=Func.OpitionsPrice(spot.TRDPRC_1.values[0].astype(float),
                            data["STRIKE_PRC"].values.astype(float),
                            spot.TRDPRC_1.values[1]/100,
                            np.array(ImpyVola)/100,
                            days)

In [ ]:
GREEKS

In [ ]:
data[['Valor','Delta','Gamma','Vega','Theta','Rho','Etas']]=pd.DataFrame([[0 for i in range(7)]], columns=['Valor','Delta','Gamma','Vega','Theta','Rho','Etas'],index=data.index)
callGreek=GREEKS[list(GREEKS.columns[GREEKS.columns.str.contains('C', regex=True)])][data.PUTCALLIND.str.contains('C', regex=True).values]
putGreek=GREEKS[list(GREEKS.columns[GREEKS.columns.str.contains('P', regex=True)])][data.PUTCALLIND.str.contains('P', regex=True).values]

callGreek.columns=['Valor','Delta','Gamma','Vega','Theta','Rho','Etas']
putGreek.columns=['Valor','Delta','Gamma','Vega','Theta','Rho','Etas']

data.loc[list(data.PUTCALLIND.str.contains('C', regex=True).values),['Valor','Delta','Gamma','Vega','Theta','Rho','Etas']]=callGreek.values
data.loc[list(data.PUTCALLIND.str.contains('P', regex=True).values),['Valor','Delta','Gamma','Vega','Theta','Rho','Etas']]=putGreek.values

In [ ]:
data[['Instrument','TRDPRC_1','STRIKE_PRC','VOL_X_PRC1','Amount','Valor','Delta','Gamma','Vega','Theta','Rho','Etas']]

## Montecarlo Simulation

In [ ]:
vola, err = ek.get_data([spot[spot["Instrument"]!="BRSELICD=CBBR"].Instrument.values[0]],
                            ['TR.Volatility5D','TR.Volatility10D',"TR.Volatility20D",
                            "TR.Volatility30D","TR.Volatility60D","TR.Volatility90D"])
vola.columns=['Instrument',5,10,20,30,60,90]
vola[[5,10,20,30,60,90]]=vola[[5,10,20,30,60,90]]/100
days=list(np.unique(days))
dists=[]
vollist=[]
for d in days:
    y_interp = scipy.interpolate.interp1d(vola.columns[1:].astype(float),vola.T[0].values[1:].astype(float))
    volaV=y_interp(d)
    Distrib=np.random.lognormal(0,(volaV/(252**0.5)*(d**0.5)), size=5000)
    Distrib=Distrib*ssp

    vollist.append(volaV)
    dists.append(Distrib)

In [ ]:
for i in range(len(dists)):
    dists[i]=dists[i][(dists[i]>ssp*0.8)&(dists[i]<ssp*1.2)]

## PnL Calculations

In [ ]:
Price=np.array(dists).reshape(np.array(dists).shape[0]*np.array(dists).shape[1])
Price.sort()

In [ ]:
(Price.min()+Price.max())/2

In [ ]:
(dists)

In [ ]:
net=[]
for k in range(len(data)):
    if data['Amount'].iloc[k]>=0:
        if "C" in data['PUTCALLIND'].iloc[k]:
            Rt= -data['STRIKE_PRC'].iloc[k]-data['TRDPRC_1'].iloc[k] +Price
            Rt[Rt<=-data['TRDPRC_1'].iloc[k]]=-data['TRDPRC_1'].iloc[k]
            Rt=Rt*abs(data['Amount'].iloc[k])
            net.append(list(Rt))
        else:
            Rt= data['STRIKE_PRC'].iloc[k] -data['TRDPRC_1'].iloc[k] -Price
            Rt[Rt<=-data['TRDPRC_1'].iloc[k]]=-data['TRDPRC_1'].iloc[k]
            Rt=Rt*abs(data['Amount'].iloc[k])
            net.append(list(Rt))

    else:
        if "C" in data['PUTCALLIND'].iloc[k]:
            Rt= data['STRIKE_PRC'].iloc[k] +data['TRDPRC_1'].iloc[k] -Price
            Rt[Rt>=data['TRDPRC_1'].iloc[k]]=data['TRDPRC_1'].iloc[k]
            Rt=Rt*abs(data['Amount'].iloc[k])
            net.append(list(Rt))
        else:
            Rt= -data['STRIKE_PRC'].iloc[k] +data['TRDPRC_1'].iloc[k] +Price
            Rt[Rt>=data['TRDPRC_1'].iloc[k]]=data['TRDPRC_1'].iloc[k]
            Rt=Rt*abs(data['Amount'].iloc[k])
            net.append(list(Rt))

tempTT=[]
for j in range(len(set(days))):
    tempT=[]
    for s in range(len(data["STRIKE_PRC"])):
        temp=Func.OpitionsPrice(Price,
                            data["STRIKE_PRC"].values.astype(float)[s],
                            spot.TRDPRC_1.values[1]/100,
                            np.array(ImpyVola[j])/100,
                            days[j])

        if "C" in data["PUTCALLIND"].values[s]:
            temp=list((temp.ValorC.values-data['TRDPRC_1'].values[s])*data['Amount'].values[s])
        else:
            temp=list((temp.ValorP.values-data['TRDPRC_1'].values[s])*data['Amount'].values[s])

        tempT.append(temp)
    tempTT.append(tempT)

In [ ]:
tempTT

## Ploting PnL

In [ ]:
def StrPL(Distrib,pay,tempTT,days,dists,data):
    
    fig = go.FigureWidget(make_subplots(shared_xaxes=True,
                                        specs=[[{"secondary_y": True}]],print_grid=False))

#     fig = go.Figure(make_subplots(shared_xaxes=True,
#                                         specs=[[{"secondary_y": True}]],print_grid=False))

    trace3 = go.Scatter(name="0 line",x=pay.index,
                           y=np.array([0 for i in Distrib]),
                           xaxis = 'x1',yaxis = 'y2',
                           line = dict(color='black', width=2, dash='dash'))
    fig.add_trace(trace3, secondary_y=False)

    trace1 = go.Scatter(name="Payoff",x=pay.index,
                           y=pay.values,xaxis = 'x1',yaxis = 'y2',
                           mode='lines',fill='tozeroy')

    fig.add_trace(trace1, secondary_y=False)

    for i in range(len(tempTT)):
        trace5 = go.Scatter(name="Price - "+str(days[i])+' Days',x=pay.index,
                               y=pd.DataFrame(tempTT[i]).sum().values,
                               xaxis = 'x1',yaxis = 'y2',
                               mode='lines')
        fig.add_trace(trace5, secondary_y=False)


    for lin,i in zip(pays.values,pays.index):
        trace4 = go.Scatter(name=data.Instrument.values[i],x=pay.index,
                               y=lin,xaxis = 'x1',yaxis = 'y2',
                               line = dict(width=2, dash='dash'))
        fig.add_trace(trace4, secondary_y=False)


    for i,j in zip(days,dists):
        trace2 = ff.create_distplot([j],[str(i)+" Days - Probabilidade"], 
                                    bin_size=.5,curve_type='normal',show_hist=False,show_rug=False)
        fig.add_trace(trace2['data'][0], secondary_y=True)
    
    fig.layout.update({'height': 800,
                       'width': 1000})
    return(fig)

pay=pd.DataFrame(net)
pay.columns=Price
pays=pay
pay=pay.sum()
fig=StrPL(Distrib,pay,tempTT,days,dists,data)
prob=round(sum(pay.values>0)/len(Distrib)*100,2)
fig
# prob=round(sum(pay.values>0)/len(Distrib)*100,2)
# plot(fig, auto_open=True)

In [ ]:

fig = go.FigureWidget(make_subplots(shared_xaxes=True,
                                    specs=[[{"secondary_y": True}]],print_grid=False))

#     fig = go.Figure(make_subplots(shared_xaxes=True,
#                                         specs=[[{"secondary_y": True}]],print_grid=False))

trace3 = go.Scatter(name="0 line",x=pay.index,
                       y=np.array([0 for i in Distrib]),
                       xaxis = 'x1',yaxis = 'y2',
                       line = dict(color='black', width=2, dash='dash'))
fig.add_trace(trace3, secondary_y=False)

trace1 = go.Scatter(name="Payoff",x=pay.index,
                       y=pay.values,xaxis = 'x1',yaxis = 'y2',
                       mode='lines',fill='tozeroy')

fig.add_trace(trace1, secondary_y=False)

for i in range(len(tempTT)):
    trace5 = go.Scatter(name="Price - "+str(days[i])+' Days',x=pay.index,
                           y=pd.DataFrame(tempTT[i]).sum().values,
                           xaxis = 'x1',yaxis = 'y2',
                           mode='lines')
    fig.add_trace(trace5, secondary_y=False)


for lin,i in zip(pays.values,pays.index):
    trace4 = go.Scatter(name=data.Instrument.values[i],x=pay.index,
                           y=lin,xaxis = 'x1',yaxis = 'y2',
                           line = dict(width=2, dash='dash'))
    fig.add_trace(trace4, secondary_y=False)


for i,j in zip(days,dists):
    trace2 = ff.create_distplot([j],[str(i)+" Days - Probabilidade"], 
                                bin_size=.5,curve_type='normal',show_hist=False,show_rug=False)
    fig.add_trace(trace2['data'][0], secondary_y=True)

fig.layout.update({'height': 800,
                   'width': 1000})

In [ ]:
min(j.tolist())

In [ ]:
trace2 = ff.create_distplot([dists[0]],[str(days[0])+" Days - Probabilidade"], 
                            bin_size=.5,curve_type='normal',show_hist=False,show_rug=False)

In [ ]:
dists[0]

In [ ]:
dists[0]

In [ ]:
type(dists[0])